# Kaggle Tabular Playground Series - Aug 2022
https://www.kaggle.com/competitions/tabular-playground-series-aug-2022<br>

In [77]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-aug-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-aug-2022/train.csv
/kaggle/input/tabular-playground-series-aug-2022/test.csv


In [78]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.linear_model import RidgeClassifier

from sklearn.ensemble import VotingClassifier

import matplotlib.pyplot as plt

random_seed = 256

import warnings
warnings.filterwarnings("ignore")

In [79]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2022/train.csv', index_col='id')
train

,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
id,,,,,,,,,,,,,,,,,,,,,
0,A,80.10,material_7,material_8,9,5,7,8,4,18.040,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26565,E,158.95,material_7,material_6,6,9,6,16,4,16.301,...,NaN,12.177,17.942,10.112,15.795,18.572,16.144,NaN,729.131,0
26566,E,146.02,material_7,material_6,6,9,10,12,8,17.543,...,11.242,14.179,20.564,10.234,14.450,14.322,13.146,16.471,853.924,0
26567,E,115.62,material_7,material_6,6,9,1,10,1,15.670,...,11.407,16.437,17.476,8.668,15.069,16.599,15.590,14.065,750.364,0


In [80]:
# Check for missing values
train.isna().sum().sum()

20273

In [81]:
# Check more detail for missing values
print('Col name', '\t\t Null values', '\t dtype')
print('=================================================')
for col in train.columns:
        print(col, '\t\t',  train[col].isnull().sum(), '\t\t', train[col].dtype)

Col name 		 Null values 	 dtype
product_code 		 0 		 object
loading 		 250 		 float64
attribute_0 		 0 		 object
attribute_1 		 0 		 object
attribute_2 		 0 		 int64
attribute_3 		 0 		 int64
measurement_0 		 0 		 int64
measurement_1 		 0 		 int64
measurement_2 		 0 		 int64
measurement_3 		 381 		 float64
measurement_4 		 538 		 float64
measurement_5 		 676 		 float64
measurement_6 		 796 		 float64
measurement_7 		 937 		 float64
measurement_8 		 1048 		 float64
measurement_9 		 1227 		 float64
measurement_10 		 1300 		 float64
measurement_11 		 1468 		 float64
measurement_12 		 1601 		 float64
measurement_13 		 1774 		 float64
measurement_14 		 1874 		 float64
measurement_15 		 2009 		 float64
measurement_16 		 2110 		 float64
measurement_17 		 2284 		 float64
failure 		 0 		 int64


In [82]:
# Check for duplicated rows
train.duplicated().sum()

0

In [83]:
# Check categorical data in train
for col in ['product_code', 'attribute_0', 'attribute_1']:
    print(col, ': \t', train[col].unique())

product_code : 	 ['A' 'B' 'C' 'D' 'E']
attribute_0 : 	 ['material_7' 'material_5']
attribute_1 : 	 ['material_8' 'material_5' 'material_6']


In [84]:
# Check categorical data in test
test = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2022/test.csv', index_col='id')

for col in ['product_code', 'attribute_0', 'attribute_1']:
    print(col, ': \t', test[col].unique())

product_code : 	 ['F' 'G' 'H' 'I']
attribute_0 : 	 ['material_5' 'material_7']
attribute_1 : 	 ['material_6' 'material_7' 'material_5']


In [85]:
# Get X data. since product_code is different between train and test, will be dropped
X_train = train.drop(['product_code', 'failure'], axis=1)
X_test = test.drop(['product_code'], axis=1)

# Get features names
feature_names = X_train.columns.values.tolist()

# Categorical encoder. since categories are shared, define one unique encoder
encoder = LabelEncoder()
encoder.fit(['material_5', 'material_6', 'material_7', 'material_8'])
for col in ['attribute_0', 'attribute_1']:
    X_train[col] = encoder.transform(X_train[col])
    X_test[col] = encoder.transform(X_test[col])

# Get y data
y_train = pd.Series(train['failure'])

In [86]:
# Imputing null values
# Create our imputer to replace missing values with the mean e.g.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(X_train)

# Impute data for train and test
X_train = imp.transform(X_train)
X_test = imp.transform(X_test)

## Models Comparisson

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessClassifier.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.RadiusNeighborsClassifier.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifier.html<br>

In [87]:
names = [
    "Logistic Regression",
    "Nearest Neighbors",
    "RBF SVM",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Extra Trees",
    "Radius Neighbors",
]

classifiers = [
    LogisticRegression(max_iter=10000, random_state=random_seed),
    KNeighborsClassifier(25),
    SVC(kernel='rbf', random_state=random_seed),
    DecisionTreeClassifier(random_state=random_seed),
    RandomForestClassifier(n_estimators=500, random_state=random_seed),
    MLPClassifier(alpha=1, max_iter=2000, random_state=random_seed),
    AdaBoostClassifier(random_state=random_seed),
    ExtraTreesClassifier(n_estimators=200, random_state=random_seed),
    RadiusNeighborsClassifier(),
]

## Cross Validation
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html<br>
https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter<br>

In [88]:
cv = StratifiedKFold(n_splits=5)

# iterate over classifiers
for name, clf in zip(names, classifiers):
    scores = cross_val_score(clf, X_train, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)
    #scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)
    print('Model:\t', name, '\t\t Score:\t', scores.mean())

Model:	 Logistic Regression 		 Score:	 0.5894338752278369
Model:	 Nearest Neighbors 		 Score:	 0.5517233777505719
Model:	 RBF SVM 		 Score:	 0.5233039605381128
Model:	 Decision Tree 		 Score:	 0.5080853611340519
Model:	 Random Forest 		 Score:	 0.534711532638587
Model:	 Neural Net 		 Score:	 0.588517473879684
Model:	 AdaBoost 		 Score:	 0.580124349706769
Model:	 Extra Trees 		 Score:	 0.5189462657607431
Model:	 Radius Neighbors 		 Score:	 nan


## Best Model

In [89]:
# Best Model
best_model = classifiers[0]
# Train best model with all train data
best_model.fit(X_train, y_train)

LogisticRegression(max_iter=10000, random_state=256)

## Submission

In [90]:
prediction = best_model.predict(X_test).squeeze()
probabilities = best_model.predict_proba(X_test)[:,1].squeeze()

row_id =  test.index.values
submission = pd.DataFrame({'id' : row_id, 'failure' : probabilities})

submission.to_csv('submission.csv', index=False)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'RadiusNeighborsClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score